# Statistika výdělků, spropitného a počtu objednávek v Klubu Betlémská
Alexandra Botková

Tato statistická práce si klade za cíl stanovit základní statistiky a ověřit dvě hypotézy, že v závislosti na denní době (den: 12.00-18.00, noc: 18:00-24.00) se liší:
1) výdělky, spropitné a počet objednávek 

2) pravděpodobnost obdržení spropitného 

Tuto pravděpodobnost vypočteme a zjistíme, zda se jedná o binomické rozdělení.

### Data
Data jsou výpisy z pokladny z kavárny/baru Klub Betlémská ze všedních dnů 11.8. - 15.8. po celou otvírací dobu, tj. od 12.00 do 24.00.

### Načtení dat
Používaná data jsou uložena v souboru betlemska.pkl, které načteme do dataframe. Pocházejí ze souboru betlemska.pdf a k jejich zpracování slouží data_extraction.py.

In [8]:
import pandas as pd

df = pd.read_pickle("betlemska.pkl")
df

,Date,Profit,Tips,Hour,TimeOfDay
12,2024-08-15 23:55:02,103.0,2.0,23,Night
15,2024-08-15 23:50:10,1.0,39.0,23,Night
16,2024-08-15 23:48:40,1.0,60.0,23,Night
17,2024-08-15 23:48:21,55.0,15.0,23,Night
18,2024-08-15 23:45:17,260.0,40.0,23,Night
...,...,...,...,...,...
923,2024-08-11 14:40:17,313.0,0.0,14,Day
924,2024-08-11 14:17:06,156.0,0.0,14,Day
925,2024-08-11 14:00:22,67.0,0.0,14,Day
926,2024-08-11 13:59:46,195.0,0.0,13,Day


### Základní statistiky
Spočteme průměry a odchylky denních výdělků, spropitného a počtu objednávek.

In [9]:
daily_summary = df.groupby(df['Date'].dt.date).agg(
    total_profit=pd.NamedAgg(column='Profit', aggfunc='sum'),
    total_tips=pd.NamedAgg(column='Tips', aggfunc='sum'),
    total_orders=pd.NamedAgg(column='Profit', aggfunc='size')
)

average_summary = daily_summary.mean()
daily_summary = daily_summary.std()

print("Průměrný denní výdělek je {:.2f} Kč s odchylkou {:.2f} Kč".format(average_summary['total_profit'], daily_summary['total_profit']))
print("Průměrné spropitné za den je {:.2f} Kč s odchylkou {:.2f} Kč".format(average_summary['total_tips'], daily_summary['total_tips']))
print("Průměrný počet objednávek za den je {:.2f} s odchylkou {:.2f}".format(average_summary['total_orders'], daily_summary['total_orders']))

Průměrný denní výdělek je 22953.00 Kč s odchylkou 7885.10 Kč
Průměrné spropitné za den je 766.00 Kč s odchylkou 243.52 Kč
Průměrný počet objednávek za den je 182.60 s odchylkou 48.89


### Hypotéza 1
Dle nulové hypotézy se průměry pozorovaných dat v závislosti na denní době neliší.

Prvně opět spočítáme průměry s odchylkami, ale objednávky rozdělíme na denní a noční.

In [10]:

daily_summary = df.groupby([df['Date'].dt.date, 'TimeOfDay']).agg(
    total_profit=pd.NamedAgg(column='Profit', aggfunc='sum'),
    total_tips=pd.NamedAgg(column='Tips', aggfunc='sum'),
    total_orders=pd.NamedAgg(column='Profit', aggfunc='size')
).reset_index()

day_summary = daily_summary[daily_summary['TimeOfDay'] == 'Day']
night_summary = daily_summary[daily_summary['TimeOfDay'] == 'Night']

mean_profit_day = day_summary['total_profit'].mean()
std_profit_day = day_summary['total_profit'].std()

mean_profit_night = night_summary['total_profit'].mean()
std_profit_night = night_summary['total_profit'].std()

mean_tips_day = day_summary['total_tips'].mean()
std_tips_day = day_summary['total_tips'].std()

mean_tips_night = night_summary['total_tips'].mean()
std_tips_night = night_summary['total_tips'].std()

mean_orders_day = day_summary['total_orders'].mean()
std_orders_day = day_summary['total_orders'].std()

mean_orders_night = night_summary['total_orders'].mean()
std_orders_night = night_summary['total_orders'].std()

print("Průměrný denní zisk během dne je {:.2f} Kč s odchylkou {:.2f} Kč.".format(mean_profit_day, std_profit_day))
print("Průměrný denní zisk během noci je {:.2f} Kč s odchylkou {:.2f} Kč.".format(mean_profit_night, std_profit_night))

print("\nPrůměrné denní spropitné během dne je {:.2f} Kč s odchylkou {:.2f} Kč.".format(mean_tips_day, std_tips_day))
print("Průměrné denní spropitné během noci je {:.2f} Kč s odchylkou {:.2f} Kč.".format(mean_tips_night, std_tips_night))

print("\nPrůměrný denní počet objednávek během dne je {:.2f} s odchylkou {:.2f}.".format(mean_orders_day, std_orders_day))
print("Průměrný denní počet objednávek během noci je {:.2f} s odchylkou {:.2f}.".format(mean_orders_night, std_orders_night))

Průměrný denní zisk během dne je 4164.20 Kč s odchylkou 1053.51 Kč.
Průměrný denní zisk během noci je 18788.80 Kč s odchylkou 7372.32 Kč.

Průměrné denní spropitné během dne je 154.00 Kč s odchylkou 56.66 Kč.
Průměrné denní spropitné během noci je 612.00 Kč s odchylkou 220.91 Kč.

Průměrný denní počet objednávek během dne je 34.00 s odchylkou 6.44.
Průměrný denní počet objednávek během noci je 148.60 s odchylkou 47.29.


Spočtené průměry nasvědčují, že sledované hodnoty jsou větší večer a chceme zjistit, zda zde vskutku je statisticky významný rozdíl. K tomu použijeme nezávislý t-test, ale předtím musíme ověřit normalitu agregovaných denních a nočních dat pomocí Shapiro-Wilkova testu.

In [11]:
from scipy.stats import shapiro

day_normality = shapiro(day_summary['total_profit'])
night_normality = shapiro(night_summary['total_profit'])

day_normality.pvalue, night_normality.pvalue

(0.8465485221184514, 0.6731689037707724)

Obě p-hodnoty naznačují, že můžeme předpokládat normalitu, tudíž můžeme použít t-test.

In [12]:
from scipy.stats import ttest_ind

t_stat_profit, p_value_profit = ttest_ind(day_summary['total_profit'], night_summary['total_profit'], equal_var=False)

t_stat_tips, p_value_tips = ttest_ind(day_summary['total_tips'], night_summary['total_tips'], equal_var=False)

t_stat_orders, p_value_orders = ttest_ind(day_summary['total_orders'], night_summary['total_orders'], equal_var=False)

print("Výsledky t-testu pro zisky:")
print("T-statistika: {:.2f}".format(t_stat_profit))
print("P-hodnota: {:.4f}".format(p_value_profit))

print("\nVýsledky t-testu pro sproppitné:")
print("T-statistika: {:.2f}".format(t_stat_tips))
print("P-hodnota: {:.4f}".format(p_value_tips))

print("\nVýsledky t-testu pro počet objednávek:")
print("T-statistika: {:.2f}".format(t_stat_orders))
print("P-hodnota: {:.4f}".format(p_value_orders))

Výsledky t-testu pro zisky:
T-statistika: -4.39
P-hodnota: 0.0108

Výsledky t-testu pro sproppitné:
T-statistika: -4.49
P-hodnota: 0.0082

Výsledky t-testu pro počet objednávek:
T-statistika: -5.37
P-hodnota: 0.0052


Všechny p-hodnoty jsou menší než 0.05, tudíž existuje statisticky významný rozdíl mezi dnem a nocí u sledovaných dat. Negativních t-statistiky potvrzují, že v noci jsou průměrné hodnoty pro zisky, spropitné a počet objednávek vyšší než ve dne.

### Hypotéza 2
Pro zjištění, zda se pravděpodobnost obdržení spropitného liší během dne a noci, použijeme chi-kvadrát test.

In [14]:
from scipy.stats import chi2_contingency

df['ReceivedTip'] = df['Tips'] > 0

contingency_table_tips = pd.crosstab(df['TimeOfDay'], df['ReceivedTip'])

chi2, p_value, dof, expected = chi2_contingency(contingency_table_tips)

print("Chi-kvadrát: {:.2f}".format(chi2))
print("P-hodnota: {:.4f}".format(p_value))

contingency_table_tips


Chi-kvadrát: 1.17
P-hodnota: 0.2796


ReceivedTip,False,True
TimeOfDay,,
Day,114,56
Night,532,211


Z důvodu vysoké p-hodnoty nezamítáme nulovou hypotézu, že neexistuje statisticky významný rozdíl mezi dnem a nocí.

### Pravděpodobnost obdržení spropitného

In [ ]:
df['ReceivedTip'] = df['Tips'] > 0

total_tips_received = df['ReceivedTip'].sum()
total_orders = df['ReceivedTip'].count()
overall_probability_tip = total_tips_received / total_orders

std_dev_overall = np.sqrt(overall_probability_tip * (1 - overall_probability_tip) / total_orders)

print("Pravděpodobnost obdržení spropitného za celý den je {:.2f} s odchylkou {:.4f}.".format(overall_probability_tip, std_dev_overall))

Pravděpodobnost obdržení spropitného za celý den je 0.29 s odchylkou 0.0151.


Pravděpodobnost vyšla 0.29. Nyní pomocí knihovní funkce ověříme nulovou hypotézu, že se jedná 
o binomické rozdělení.

In [ ]:
from scipy.stats import binomtest

binom_test_result_two_tailed = binomtest(total_tips_received, total_orders, 0.29)

binom_test_result_two_tailed.pvalue

0.8840263196145653

Nulovou hypotézu nezamítáme, tedy pozorované frekvence spropitného jsou v souladu s binomickým rozdělením s pravděpodobností 0.29.